This file uses imported functions from the public_psm_commonfunctions file. The functions file contains the **weapon_conversion_ key** as well as the following functions: 
- **process_weapon**
- **watch_from_squad_desc**
- **generate_features**
- **get_rolling_count**

The code uses the following packages: 
- pandas
- numpy
- datetime

In [ ]:
# Import packages/functions
from public_psm_commonfunctions import weapon_conversion_key, process_weapon, \
     watch_from_squad_desc, generate_features
import pandas as pd
import numpy as np
from datetime import datetime

The formatting code allows for easier comprehension of the dataframes.

In [ ]:
# formatting
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', '{:.2f}'.format)

The dataset was imported from the Seattle Police Department website. More information about the dataset can be seen here:<br>
https://data.seattle.gov/Public-Safety/Terry-Stops/28ny-9ts8. <br>
The original dataset is imported and named full_df, which will be the dataframe that will yield the final result.

In [ ]:
# Import public dataset
full_df = pd.read_csv("Terry_Stops.csv")

The following code removes any rows with null/invalid "Subject Perceived Race" values and any rows with null/invalid "Precinct" values. The specific values that are considered null/invalid can be seen below:

Subject Perceived Race:
- 'Unknown', 'DUPLICATE', '-', 'Null'

Precinct:
- 'Unknown', 'DUPLICATE', '-', 'Null', 'FK ERROR'


In [ ]:
# Drop rows with non determinant or null race
non_determinant_race = ['Unknown', 'DUPLICATE', '-','Null']
full_df = full_df[~full_df['Subject Perceived Race'].isin(non_determinant_race)]
full_df = full_df[full_df['Subject Perceived Race'].notnull()]
print('Dropped rows with non determinant or null race information', full_df.shape)

In [ ]:
# Drop rows with no precinct information
non_determinant_precinct = ['Unknown', 'DUPLICATE', '-','Null','FK ERROR']
full_df = full_df[~full_df['Precinct'].isin(non_determinant_precinct)]
full_df = full_df[full_df['Precinct'].notnull()]
print('Dropped rows with no precinct information', full_df.shape)

The code below generates a weapons feature that matches with the following values: 
- 1 meaning that there was a weapon involved.
- 0 meaning that there was no weapon involved.

The code uses the "Terry Stop ID" column as a key to generate the weapons feature using the **weapon_conversion_key** and the function **process_weapon** based on the column "weapon_type". After the conversion, the column is merged with the original dataframe.

In [ ]:
# Weapon feature engineering
feature = 'Weapon Type'
full_df['weapon_type'] = full_df['Weapon Type'].map(weapon_conversion_key)
unique_keys = ['Terry Stop ID']
weapons = full_df.groupby(unique_keys).agg(
    weapons_list = (feature, list), 
    weapon_type=('weapon_type', max)
).reset_index()
weapons['weapons_list'] = weapons['weapons_list'].apply(lambda x: process_weapon(x))
weapons['weapon_count'] = weapons['weapons_list'].apply(lambda x: len(x))
weapons = weapons.drop('weapons_list', axis=1)
full_df = full_df.drop([feature, 'weapon_type'], axis=1).merge(weapons, how='left', on=unique_keys)
full_df = full_df.drop_duplicates()
full_df.shape
full_df.dtypes
print('Feature: weapons', full_df.shape)

Using the datetime package, "Reported Date" and "Reported Time" are combined and converted to a single column named "observation_datetime_d". This column is now type datetime64.
Then, the following columns are generated based on the "observation_datetime_d" column:
- "observation_year_d"
- "observation_month_d"
- "observation_day_d"
- "observation_week_d"
- "observation_time_d"
- "observation_week_of_month_d"

In [ ]:
# Convert date and time to pandas datetime object and pandas timedelta object
full_df['observation_datetime_d'] = pd.to_datetime(full_df['Reported Date']) + pd.to_timedelta(full_df['Reported Time'])
full_df = full_df.drop(
    columns = ['Reported Time', 'Reported Date']
)

print('Bad data with null date:', full_df[full_df['observation_datetime_d'].isnull()].shape[0])
full_df = full_df[(full_df['observation_datetime_d'].notnull())]
full_df['observation_year_d'] = full_df['observation_datetime_d'].dt.year.astype(int)
full_df['observation_month_d'] = full_df['observation_datetime_d'].dt.month.astype(int)
full_df['observation_day_d'] = full_df['observation_datetime_d'].dt.day
full_df['observation_week_d'] = full_df['observation_datetime_d'].dt.isocalendar().week
full_df['observation_time_d'] = full_df['observation_datetime_d'].dt.time
full_df['observation_week_of_month_d'] = full_df['observation_day_d'].apply(lambda x: x//7)
full_df = full_df.drop_duplicates()
print('Feature: abstracted time', full_df.shape)

The subsequent code generates the features for precinct and watch. 

Column "watch_d" is generated using the **watch_from_squad_desc** function, where the input columns are "Officer Squad" and "observation_datetime_d". 

Column "Precinct" is generated by utilizing an if statement. 
- If the cell ends with ' PCT', remove it from the cell. 
- Capatalize every letter in each cell.

Column "Precinct_watch_d" is generated by using an if statement:
- If the cell of "watch_d" contains '1ST', '2ND', or '3RD', then combine "Precinct" and "watch_d"
- If the cell of "watch_d" does not contain any of '1ST', '2ND', or '3RD', then only use "watch_d"


In [ ]:
# Generating features for precinct and watch
full_df['watch_d'] = full_df.apply(lambda x: watch_from_squad_desc(x['Officer Squad'], x['observation_datetime_d']), axis=1)
full_df['Precinct'] = full_df['Precinct'].apply(lambda x: x[:-4].upper() if x.endswith(' PCT') else x.upper())
full_df['Precinct_watch_d'] = full_df.apply(
    lambda x: x['Precinct'] + '_' + x['watch_d'] if x['watch_d'] in ['1ST', '2ND', '3RD'] else x['watch_d'], 
    axis=1   
)
print('Generating features for precinct and watch', full_df.shape)

In the original dataset, cells where Subject ID = -1 represents when the Subject's ID was not verified in the dataset. Therefore, these values can be replaced by a null value.

In [ ]:
# Replace instances where Subject ID = -1 with Null.
full_df['Subject ID'] = pd.to_numeric(full_df['Subject ID'], errors = 'coerce')
full_df['Subject ID'] = full_df['Subject ID'].replace(-1,np.nan)

The dataframe used for the feature engineering process is named subjects_features_df. These are the columns used in the dataframe: 
- "Terry Stop ID"
- "Subject ID"
- "observation_datetime_d"
- "Precinct"
- "Officer ID"
- "weapon_type" 
- "Subject Perceived Race" 
- "Frisk Flag"

Duplicates of "Terry Stop ID" are dropped and the dataframe is sorted based on the column "observation_datetime_d". The features are all generated for five different lookback periods:
- 3 months
- 6 months
- 1 year
- 2 years
- 3 years

In [ ]:
# Person features - pre-processing                  
cols_for_subject_features = [
    'Terry Stop ID','Subject ID', 'observation_datetime_d',
    'Precinct', 'Officer ID', 'weapon_type', 'Subject Perceived Race', 'Frisk Flag'
]
full_df.drop_duplicates(subset = "Terry Stop ID",inplace = True)
subject_feature_df = full_df[cols_for_subject_features]
subject_feature_df = subject_feature_df.sort_values('observation_datetime_d')
lookback_periods = ['3M', '6M', '1Y', '2Y', '3Y']

The following cells all generate features for a specific case. Depending on the case, a groupby key is chosen. Functions **generate_features** and **get_rolling_count** are used in order to determine the count of terry stops detected in the case. There are five different counts generated for each feature as there are five different lookback periods to input into the .rolling window. 
- Feature: Number of times person stopped.
    - The column used for the groupby key is "Subject ID". <br>
    
- Feature: Number of times person stopped in precinct.
    - The columns used for the groupby key are "Precinct" and "Subject ID". <br>
    
- Feature: Number of times particular race stopped in precinct, in terms of absolute count and percentage.
    - The columns used for the groupby key are "Precinct" and "Subject Perceived Race". <br>
    
- Feature: Number of stops in precinct.
    - The column used for the groupby key is "Precinct". <br>
    
- Feature: Percentage of race stopped in precinct.
    - To generate this feature, features "n_race_stopped_in_precinct" and "n_stops_in_precinct" are divided to get the percentage.    <br>
    
- Feature: Number of times particular race stopped by officer.
    - The columns used for the groupby key are "Officer ID" and "Subject Perceived Race". <br>
    
- Feature: Number of stops by officer.
    - The column used for the groupby key is "Officer ID". <br>
    
- Feature: % race stopped by officer.
    - To generate this feature, features "n_race_stopped_by_officer" and "n_stops_by_officer" are divided to get the percentage.<br>
    
- Feature: Number of times person stopped by particular officer.
    - The columns used for the groupby key are "Officer ID" and "Subject ID". <br>
    
- Feature: Number of times person stopped by particular officer and frisked.
    - The columns used for the groupby key are "Officer ID", "Frisk Flag", and "Subject ID". <br>
    

In [ ]:
# Person features - # times person stopped
groupby_key = ['Subject ID']
feature_name = 'n_subject_stopped'
for period in lookback_periods:    
    subject_feature_df = generate_features(subject_feature_df, groupby_key, feature_name, period)
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - # times person stopped in precinct
groupby_key=['Precinct', 'Subject ID']
feature_name = 'n_subject_stopped_in_precinct'
for period in lookback_periods:
    subject_feature_df = generate_features(subject_feature_df, groupby_key, feature_name, period)
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - # times particular race stopped in precinct (absolute count and in % terms)
groupby_key=['Precinct', 'Subject Perceived Race']
feature_name = 'n_race_stopped_in_precinct'
for period in lookback_periods:
    subject_feature_df = generate_features(subject_feature_df, groupby_key, feature_name, period)
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - # stops in precinct
groupby_key=['Precinct']
feature_name = 'n_stops_in_precinct'
for period in lookback_periods:
    subject_feature_df = generate_features(subject_feature_df, groupby_key, feature_name, period)
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - % race stopped in precinct
feature_name='percent_race_stopped_in_precinct'
for period in lookback_periods:
    subject_feature_df[f'{feature_name}_{period}'] = \
        subject_feature_df[f'n_race_stopped_in_precinct_{period}']/subject_feature_df[f'n_stops_in_precinct_{period}']
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - # times particular race stopped by officer
groupby_key=['Officer ID', 'Subject Perceived Race']
feature_name='n_race_stopped_by_officer'
for period in lookback_periods:
    subject_feature_df = generate_features(subject_feature_df, groupby_key, feature_name, period)
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - # stops by officer
groupby_key=['Officer ID']
feature_name = 'n_stops_by_officer'
for period in lookback_periods:
    subject_feature_df = generate_features(subject_feature_df, groupby_key, feature_name, period)
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - % race stopped by officer
feature_name='percent_race_stopped_by_officer'
for period in lookback_periods:
    subject_feature_df[f'{feature_name}_{period}'] = \
        subject_feature_df[f'n_race_stopped_by_officer_{period}']/subject_feature_df[f'n_stops_by_officer_{period}']
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - # times person stopped by particular officer
groupby_key=['Officer ID', 'Subject ID']
feature_name='n_subject_stopped_by_officer'
for period in lookback_periods:
    subject_feature_df = generate_features(subject_feature_df, groupby_key, feature_name, period)
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - # times person stopped by particular officer & frisked
groupby_key=['Officer ID', 'Frisk Flag', 'Subject ID']
feature_name='n_subject_stopped_officer_frisk'
for period in lookback_periods:
    subject_feature_df = generate_features(subject_feature_df, groupby_key, feature_name, period)
    print(f'Feature: {feature_name}_{period}')

The first ten features for each lookback period have been generated. The code below merges the first ten features with full_df.

In [ ]:
# merge dfs
full_df = full_df.merge(subject_feature_df, on=cols_for_subject_features, how='left')

For the last two features, some extra steps were required. Another dataframe, named \_subject_feature_df, is created. This dataframes only contains rows instances where "weapon_type" = 1. The following features use this new data frame:
- Feature: Number of times person stopped in precinct with weapon involved.
    - The columns used for the groupby key are "Precinct" and "Subject ID".
- Feature: Number of times person stopped by officer with weapon involved.
    - The columns used for the groupby key are "Officer ID" and "Subject ID".

After the remaining two features have been generated for each lookback period, the features are merged onto full_df. 

In [ ]:
# Person features - # times person stopped in precinct with weapon involved
_subject_feature_df = subject_feature_df[subject_feature_df['weapon_type']==1]
groupby_key=['Precinct', 'Subject ID']
feature_name='n_subject_stopped_precinct_weapon'
feature_set = []
for period in lookback_periods:
    _subject_feature_df = generate_features(_subject_feature_df, groupby_key, feature_name, period)
    feature_set.append(f'{feature_name}_{period}')
    print(f'Feature: {feature_name}_{period}')

In [ ]:
# Person features - # times person stopped by officer with weapon involved
groupby_key=['Officer ID', 'Subject ID']
feature_name='n_subject_stopped_by_officer_weapon'
for period in lookback_periods:
    _subject_feature_df = generate_features(_subject_feature_df, groupby_key, feature_name, period)
    feature_set.append(f'{feature_name}_{period}')
    print(f'Feature: {feature_name}_{period}')
full_df = full_df.merge(
    _subject_feature_df[['Terry Stop ID','Subject ID'] + feature_set], 
    on=['Terry Stop ID','Subject ID'], 
    how='left'
)

The following code changes rows where Subject ID is not null such that if the features in that row are null, they are replaced with the value 0. Once that is done, full_df is exported to 'model_features.csv'.

In [ ]:
# in rows where Subject ID is not null, if the features in that row are null, replace them with 0
temp = full_df['Subject ID'].notnull()
full_df.loc[temp] = full_df.loc[temp].fillna(0)

In [ ]:
# Export CSV of final result
full_df.to_csv('model_features.csv') 
print(f'Job done!')    

The original dataset found in the Seattle Police Department website can be seen below.

In [ ]:
originalDataset = pd.read_csv("Terry_Stops.csv")
originalDataset

The final dataset, including all of the features, is shown here.

In [ ]:
finalDataset = full_df
finalDataset